In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import os
import numpy as np
import pandas as pd
import parse
from model import *

In [5]:
# replay = os.path.join( "train", os.listdir("train")[0])
# data = parse.parse_replay_file(replay)

replay_data = parse.parse_replay_folder("train", max_files=2)

  0%|          | 0/4 [00:00<?, ?it/s]

Parsing file 1
Load Replay: train/ts2018-halite-3-gold-replays_replay-20181009-142856+0000-1539095317-32-32
Parsing data for ['tsadmiral v1', 'anglee v2', 'jon-stumpf v6', 'Daniel_Kukiela v5']
Load Basic Information for {'energy': 5000, 'entities': [], 'factory_location': {'x': 8, 'y': 8}, 'name': 'tsadmiral v1', 'player_id': 0}
Load Cell Information



 25%|██▌       | 1/4 [00:08<00:25,  8.66s/it]

Load Player Ships
Load Other Player Ships
Load Droppoff Information
Load Basic Information for {'energy': 5000, 'entities': [], 'factory_location': {'x': 23, 'y': 8}, 'name': 'anglee v2', 'player_id': 1}
Load Cell Information



 50%|█████     | 2/4 [00:17<00:17,  8.75s/it]

...

Load Player Ships
Load Other Player Ships
Load Droppoff Information
Parsing file 2
Load Replay: train/ts2018-halite-3-gold-replays_replay-20181009-145449+0000-1539096776-64-64
Parsing data for ['tsadmiral v1', 'revoklaw v9']
Load Basic Information for {'energy': 5000, 'entities': [], 'factory_location': {'x': 16, 'y': 32}, 'name': 'tsadmiral v1', 'player_id': 0}
Load Cell Information



 50%|█████     | 1/2 [00:43<00:43, 43.70s/it]

Load Player Ships
Load Other Player Ships
Load Droppoff Information
Load Basic Information for {'energy': 5000, 'entities': [], 'factory_location': {'x': 47, 'y': 32}, 'name': 'revoklaw v9', 'player_id': 1}
Load Cell Information



100%|██████████| 2/2 [02:04<00:00, 51.36s/it]

Load Player Ships
Load Other Player Ships
Load Droppoff Information


In [8]:
game_states = []
for game in data:
    for player_data in game:
        game_states.append(HaliteModel.process_game_states(player_data))
        
game_states = np.array(game_states)

Processing Game States
Processing Game States
Processing Game States
Processing Game States
Processing Game States
Processing Game States


In [11]:
game_states.shape

(6,)

In [30]:
m = HaliteModel(data=data)
features, labels = [], []

"""
f_checkpoint = open("features.npy", "a")
l_checkpoint = open("labels.npy", "a")
"""

for i, game in enumerate(game_states):
    print(f"Game {i+1}/{len(game_states)}")
    f, l = m.generate_training_data(game)
    
    """
    np.save(f_checkpoint, np.array(f))
    np.save(l_checkpoint, np.array(l))
    """
    
    features.extend(f)
    labels.extend(l)
    

Generating Training Data


HBox(children=(IntProgress(value=0, max=2796), HTML(value='')))


Generating Training Data


HBox(children=(IntProgress(value=0, max=1253), HTML(value='')))


Generating Training Data


...

HBox(children=(IntProgress(value=0, max=2614), HTML(value='')))


Generating Training Data


HBox(children=(IntProgress(value=0, max=6492), HTML(value='')))


Generating Training Data


HBox(children=(IntProgress(value=0, max=6131), HTML(value='')))

In [31]:
features = np.array(features)
labels = np.array(labels)

features.shape, labels.shape

((60160, 78), (60160,))

In [32]:
np.save("features", features)
np.save("labels", labels)

In [45]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [178]:
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.65)

No noticeable improvement found when regularizing.

In [73]:
model = MLPClassifier(solver="sgd")
model.fit(X_train, y_train)

/home/jacobgdt/anaconda3/envs/cling/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='sgd', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [74]:
preds = model.predict(X_test)
accuracy_score(preds, y_test)

0.6204224631751227

In [195]:
import pickle
with open("Agent.nn", 'wb') as f:
    pickle.dump(model, f)

In [91]:
pd.DataFrame({"y":y_train}).y.value_counts()

0    12609
2     2157
1     2139
4     2083
3     2068
Name: y, dtype: int64

In [86]:
from sklearn.utils import resample

https://elitedatascience.com/imbalanced-classes

In [120]:
df = None
features.shape

(60160, 78)

In [179]:
df = pd.DataFrame(X_train) # pd.merge([pd.DataFrame(features), pd.Series(labels.T)])
df["y"] = y_train
print(df.shape)
df.y.value_counts()

(21056, 79)


0    12659
4     2144
1     2106
3     2079
2     2068
Name: y, dtype: int64

In [180]:
majority = df[df.y == 0]
minority = df[df.y != 0]

min_upsampled = resample(minority, replace=True, n_samples=len(majority)*2)

upsampled = pd.concat([majority.sample(len(majority)//2), min_upsampled])
upsampled.y.value_counts()

4    6501
1    6383
0    6329
3    6256
2    6178
Name: y, dtype: int64

In [181]:
upsampled_y = upsampled.y
upsampled_X = upsampled.drop("y", axis=1)

In [194]:
model = MLPClassifier(solver="sgd") 

model.hidden_layer_sizes = (150,)

model.fit(upsampled_X, upsampled_y)
model.score(X_test, y_test)

/home/jacobgdt/anaconda3/envs/cling/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.44606689852700493

In [191]:
pd.Series(model.predict(X_test)).value_counts()

0    10871
2     7243
4     7213
3     7046
1     6731
dtype: int64